In [8]:
import torch.nn as nn
import torch
import torchvision
import torch.nn.init as init

In [12]:
class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet,self).__init__()
        
        #Feature module
        self.model0=OpenPose_Feature()
        
        #Stage module
        #PAFs (part affinity fields) side
        self.model1_1=make_OpenPose_block('block1_1')
        self.model2_1=make_OpenPose_block('block2_1')
        self.model3_1=make_OpenPose_block('block3_1')
        self.model4_1=make_OpenPose_block('block4_1')
        self.model5_1=make_OpenPose_block('block5_1')
        self.model6_1=make_OpenPose_block('block6_1')
        
        #confidence heatmap side
        self.model1_2=make_OpenPose_block('block1_2')
        self.model2_2=make_OpenPose_block('block2_2')
        self.model3_2=make_OpenPose_block('block3_2')
        self.model4_2=make_OpenPose_block('block4_2')
        self.model5_2=make_OpenPose_block('block5_2')
        self.model6_2=make_OpenPose_block('block6_2')
        
    def forward(self,x):
        #順伝搬定義
        
        #Feature module
        out1=self.model0(x)
        
        #stage1
        out1_1=self.model1_1(out1) #PAFs side
        out1_2=self.model1_2(out1) #confidence heatmap side
        
        #stage2
        out2=torch.cat([out1_1,out1_2,out1],1) #次元1のチャネルで結合
        out2_1=self.model2_1(out2)
        out2_2=self.model2_2(out2)
        
        #stage3
        out3=torch.cat([out2_1,out2_2,out1],1)
        out3_1=self.model3_1(out3)
        out3_2=self.model3_2(out3)
        
        #stage4
        out4=torch.cat([out3_1,out3_2,out1],1)
        out4_1=self.model4_1(out4)
        out4_2=self.model4_2(out4)
        
        #stage5
        out5=torch.cat([out4_1,out4_2,out1],1)
        out5_1=self.model5_1(out5)
        out5_2=self.model5_2(out5)
        
        #stage6
        out6=torch.cat([out5_1,out5_2,out1],1)
        out6_1=self.model6_1(out6)
        out6_2=self.model6_2(out6)
        
        #損失の計算用に各stageの結果を格納
        saved_for_loss=[]
        saved_for_loss.append(out1_1)
        saved_for_loss.append(out1_2)
        saved_for_loss.append(out2_1)
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)
        
        #最終的なPAFsのout6_1とconfidence heatmapのout6_2，そして，損失計算用に各ステージでのPAFsとheatmapを格納したsaved_for_lossを出力
        #out6_1 : torch.Size([minibatch, 38, 46, 46])
        #out6_2 : torch.Size([minibatch, 19, 46, 46])
        #saved_for_loss : [out1_1, out1_2, ... , out6_2]
        
        return (out6_1, out6_2), saved_for_loss

In [13]:
class OpenPose_Feature(nn.Module):
    def __init__(self):
        super(OpenPose_Feature,self).__init__()
        
        #VGG-19の最初10個の畳み込みを使用
        #初めて実行する際はモデルの重みパラメタをダウンロードするため実行に時間がかかる
        vgg19=torchvision.models.vgg19(pretrained=True)
        model={}
        model['block0']=vgg19.features[0:23] #vgg19の最初の10畳み込み層まで
        
        #残りは新たな畳み込み層を2つ用意
        model['block0'].add_module("23", torch.nn.Conv2d(
            512,256,kernel_size=3,stride=1,padding=1))
        model['block0'].add_module("24", torch.nn.ReLU(inplace=True))
        model['block0'].add_module("25", torch.nn.Conv2d(
            256,128,kernel_size=3,stride=1,padding=1))
        model['block0'].add_module("26", torch.nn.ReLU(inplace=True))
        
        self.model=model['block0']
        
    def forward(self,x):
        outputs=self.model(x)
        return outputs

In [14]:
def make_OpenPose_block(block_name):
    #コンフィギュレーション変数からOpenPoseのStageモジュールのblockを作成．nn.Moduleではなく，nn.Sequential
    
    #1.コンフィギュレーションの辞書変数blocksを作成し，ネットワークを生成させる
    #最初に全パターンの辞書を用意し，引数block_nameのみを生成する
    blocks={}
    #stage1
    blocks['block1_1']=[{'conv5_1_CPM_L1':[128,128,3,1,1]},
                        {'conv5_2_CPM_L1':[128,128,3,1,1]},
                        {'conv5_3_CPM_L1':[128,128,3,1,1]},
                        {'conv5_4_CPM_L1':[128,512,1,1,0]},
                        {'conv5_5_CPM_L1':[512,38,1,1,0]}]
    
    blocks['block1_2']=[{'conv5_1_CPM_L2':[128,128,3,1,1]},
                        {'conv5_2_CPM_L2':[128,128,3,1,1]},
                        {'conv5_3_CPM_L2':[128,128,3,1,1]},
                        {'conv5_4_CPM_L2':[128,512,1,1,0]},
                        {'conv5_5_CPM_L2':[512,19,1,1,0]}]
    
    #stage2-6
    for i in range(2,7):
        blocks['block%d_1' % i]=[
            {'Mconv1_stage%d_L1' % i:[185,128,7,1,3]},
            {'Mconv2_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv3_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv4_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv5_stage%d_L1' % i:[128,128,7,1,3]},
            {'Mconv6_stage%d_L1' % i:[128,128,1,1,0]},
            {'Mconv7_stage%d_L1' % i:[128,38,1,1,0]}
        ]
        blocks['block%d_2' % i]=[
            {'Mconv1_stage%d_L2' % i:[185,128,7,1,3]},
            {'Mconv2_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv3_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv4_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv5_stage%d_L2' % i:[128,128,7,1,3]},
            {'Mconv6_stage%d_L2' % i:[128,128,1,1,0]},
            {'Mconv7_stage%d_L2' % i:[128,19,1,1,0]}
        ]
        
    #引数block_nameのコンフィギュレーション辞書を取り出す
    cfg_dict=blocks[block_name]
    
    #2.コンフィギュレーション内容をリスト変数layersに格納
    layers=[]
    
    #0番目から最後の層までを作成
    for i in range(len(cfg_dict)):
        for k, v in cfg_dict[i].items():
            if 'pool' in k:
                layers+=[nn.MaxPool2d(kernel_size=v[0],stride=v[1],padding=v[2])]
            else:
                conv2d=nn.Conv2d(in_channels=v[0],out_channels=v[1],kernel_size=v[2],stride=v[3],padding=v[4])
                layers+=[conv2d,nn.ReLU(inplace=True)]
                
    #3.layersをSequentialにする
    #ただし，最後にReLUはいらないのでその手前までを使用
    net=nn.Sequential(*layers[:-1])
    
    #4.初期化関数の設定をし，畳み込み層を初期化する
    def _initialize_weights_norm(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                init.normal_(m.weight,std=0.01)
                if m.bias is not None:
                    init.constant_(m.bias,0.0)
                    
    net.apply(_initialize_weights_norm)
    
    return net

In [22]:
#モデルの定義
net = OpenPoseNet()
net.train()

#ダミーデータ作成
batch_size=2
dummy_img=torch.rand(batch_size,3,368,368)

#calc
outputs=net(dummy_img)
print(outputs)

((tensor([[[[ 4.9497e-06, -2.6408e-05,  1.9092e-07,  ...,  2.3468e-05,
            3.5259e-05, -1.3409e-05],
          [ 3.1347e-05,  1.5046e-05, -9.6169e-06,  ...,  1.3223e-05,
            9.5709e-06, -3.8902e-05],
          [ 1.5400e-05,  3.3331e-05,  1.2672e-05,  ...,  2.4362e-05,
            4.0147e-05, -7.6882e-06],
          ...,
          [-3.6321e-05, -1.5452e-05, -4.2119e-05,  ...,  2.6669e-06,
           -3.1031e-05, -3.4148e-05],
          [-3.4184e-05,  3.0868e-05, -6.5091e-06,  ..., -2.6682e-06,
           -4.5122e-05, -4.5387e-05],
          [-2.5247e-05, -1.8498e-05, -1.0868e-05,  ..., -2.6317e-05,
           -3.4307e-05, -1.9283e-05]],

         [[-3.3250e-05, -2.3710e-05, -2.9936e-05,  ..., -1.7179e-06,
           -9.2023e-07, -3.2798e-05],
          [-2.6047e-05, -2.5196e-05, -7.4265e-05,  ..., -2.5980e-05,
           -2.5080e-05, -2.5278e-05],
          [-1.3724e-05, -2.4680e-05, -4.5650e-05,  ..., -5.5724e-05,
           -5.1102e-05, -4.1889e-05],
          ...,
   